In [ ]:
import pickle
import numpy as np

file_path = "../data/raw/s01.dat"

with open(file_path, "rb") as f:
    data = pickle.load(f, encoding="latin1")

print("Keys:", data.keys())
print("Data shape:", data["data"].shape)
print("Labels shape:", data["labels"].shape)
